In [1]:
import daft

In [2]:
daft.context.set_runner_ray(
    # Insert address of cluster here
    # address="ray://my-cluster:10001"
)

DaftContext(_daft_execution_config=<daft.daft.PyDaftExecutionConfig object at 0x10965b630>, _daft_planning_config=<daft.daft.PyDaftPlanningConfig object at 0x10965b590>, _runner_config=_RayRunnerConfig(address=None, max_task_backlog=None), _disallow_set_runner=True, _runner=None)

In [3]:
import daft
from pyiceberg.catalog import load_catalog

catalog = load_catalog()
table = catalog.load_table("nyc_taxi.fhvhv")

In [4]:
df = daft.read_iceberg(table)
df.count_rows()

2024-03-20 10:27:50,151	INFO worker.py:1724 -- Started a local Ray instance.


ScanWithTask-Aggregate [Stage:2]:   0%|          | 0/1 [00:00<?, ?it/s]

ReduceMerge-Aggregate-Project-Project [Stage:1]:   0%|          | 0/1 [00:00<?, ?it/s]

212416083

In [5]:
aggregated_df = (
    df
    .groupby(daft.col("pickup_datetime").cast(daft.DataType.date()))
    .sum(
        df["trip_time"],
        (df["base_passenger_fare"] + df["tolls"]).alias("fare_plus_tolls")
    )
    .sort("pickup_datetime")
)
aggregated_df.collect()

ScanWithTask-Project-Aggregate-FanoutHash [Stage:5]:   0%|          | 0/1 [00:00<?, ?it/s]

ReduceMerge-Aggregate-Project [Stage:4]:   0%|          | 0/1 [00:00<?, ?it/s]

Sample [Stage:6]:   0%|          | 0/1 [00:00<?, ?it/s]

ReduceToQuantiles [Stage:7]:   0%|          | 0/1 [00:00<?, ?it/s]

FanoutRange [Stage:8]:   0%|          | 0/1 [00:00<?, ?it/s]

ReduceMergeAndSort [Stage:3]:   0%|          | 0/1 [00:00<?, ?it/s]

pickup_datetimeDate,trip_timeInt64,fare_plus_tollsFloat32
2022-01-01,476689058,11388624
2022-01-02,357439522,8020001.5
2022-01-03,394632767,8248339
2022-01-04,386754975,7875813
2022-01-05,397460543,7890997.5
2022-01-06,424654888,8332166.5
2022-01-07,456685599,9474772
2022-01-08,501159393,10046370
